In [10]:
!gcloud components update -y

ERROR: (gcloud.components.update) unrecognized arguments: -y 

To search the help text of gcloud commands, run:
  gcloud help -- SEARCH_TERMS


In [36]:
import os
PROJECT_ID = "felipe-sandbox"
os.environ["PROJECT_ID"] = PROJECT_ID
print(os.environ["PROJECT_ID"])
APP_NAME = "pytorch-mnist-example"
os.environ["APP_NAME"] = APP_NAME


felipe-sandbox


In [2]:
%%bash
git clone https://github.com/pytorch/serve.git \
  --branch=v0.3.0 \
  --depth=1

cd serve/examples/image_classifier/mnist

Cloning into 'serve'...
Note: checking out 'ccff97780628a01ab3b47d835d8e9dd01377d881'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>



In [15]:
%%bash
gcloud artifacts repositories create pytorch-mnist-example \
 --repository-format=docker \
 --location=us-central1

Create request issued for: [pytorch-mnist-example]
Waiting for operation [projects/felipe-sandbox/locations/us-central1/operations/533c7b01-9c22-4974-a5f9-30d98cdde737] to complete...
......done.
Created repository [pytorch-mnist-example].


In [24]:
%%bash
cat > Dockerfile <<END
FROM pytorch/torchserve:0.3.0-cpu

COPY mnist.py mnist_cnn.pt mnist_handler.py /home/model-server/

USER root
RUN printf "\nservice_envelope=json" >> /home/model-server/config.properties
RUN printf "\ninference_address=http://0.0.0.0:7080" >> /home/model-server/config.properties
RUN printf "\nmanagement_address=http://0.0.0.0:7081" >> /home/model-server/config.properties
USER model-server
USER model-server


# expose health and prediction listener ports from the image
EXPOSE 7080
EXPOSE 7081

RUN torch-model-archiver \
  --model-name=mnist \
  --version=1.0 \
  --model-file=/home/model-server/mnist.py \
  --serialized-file=/home/model-server/mnist_cnn.pt \
  --handler=/home/model-server/mnist_handler.py \
  --export-path=/home/model-server/model-store

CMD ["torchserve", \
     "--start", \
     "--ts-config=/home/model-server/config.properties", \
     "--models", \
     "mnist=mnist.mar"]
END

In [37]:
%%bash
docker build \
  --tag=us-central1-docker.pkg.dev/$PROJECT_ID/$APP_NAME/serve-mnist \
  .

Sending build context to Docker daemon  56.66MB
Step 1/12 : FROM pytorch/torchserve:0.3.0-cpu
 ---> d7dbb9636522
Step 2/12 : COPY mnist.py mnist_cnn.pt mnist_handler.py /home/model-server/
 ---> Using cache
 ---> 51a673abd201
Step 3/12 : USER root
 ---> Using cache
 ---> 64289076516c
Step 4/12 : RUN printf "\nservice_envelope=json" >> /home/model-server/config.properties
 ---> Using cache
 ---> b438f3292c50
Step 5/12 : RUN printf "\ninference_address=http://0.0.0.0:7080" >> /home/model-server/config.properties
 ---> Using cache
 ---> e029e58a67de
Step 6/12 : RUN printf "\nmanagement_address=http://0.0.0.0:7081" >> /home/model-server/config.properties
 ---> Using cache
 ---> 6a799e02f151
Step 7/12 : USER model-server
 ---> Using cache
 ---> f958f7a15e8c
Step 8/12 : USER model-server
 ---> Using cache
 ---> f80e468887f9
Step 9/12 : EXPOSE 7080
 ---> Using cache
 ---> 97bb2a323001
Step 10/12 : EXPOSE 7081
 ---> Using cache
 ---> 391ceab33826
Step 11/12 : RUN torch-model-archiver   --model

In [39]:
%%bash
docker run -d -p 7080:7080 --name=local_mnist2 \
  us-central1-docker.pkg.dev/$PROJECT_ID/$APP_NAME/serve-mnist

ed4a0b44ae21d3867f0a4a693166b795c2ec6ac5d4d399cb7c130935a8139ffd


In [40]:
!curl localhost:7080/ping

{
  "status": "Healthy"
}


In [41]:
%%bash
cat > instances.json <<END
{
  "instances": [
    {
      "data": {
        "b64": "$(base64 --wrap=0 test_data/3.png)"
      }
    }
  ]
}
END

curl -X POST \
  -H "Content-Type: application/json; charset=utf-8" \
  -d @instances.json \
  localhost:7080/predictions/mnist

{"predictions": [3]}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   422  100    20  100   402    588  11823 --:--:-- --:--:-- --:--:-- 12411


In [43]:
!docker stop local_mnist2

local_mnist2


In [44]:
%%bash
gcloud auth configure-docker us-central1-docker.pkg.dev
docker push us-central1-docker.pkg.dev/$PROJECT_ID/$APP_NAME/serve-mnist

Using default tag: latest
The push refers to repository [us-central1-docker.pkg.dev/felipe-sandbox/pytorch-mnist-example/serve-mnist]
175e0b2fdf31: Preparing
0dee4818e1b0: Preparing
ffa1fd9a60c2: Preparing
99baa2214840: Preparing
eaaf0853d3af: Preparing
5f70bf18a086: Preparing
4f22e947a5a8: Preparing
e681618568c0: Preparing
711c3c5d494b: Preparing
ac4e7c2ae042: Preparing
44486128a8c5: Preparing
c761bb163c21: Preparing
d54e4a309a82: Preparing
fe6d8881187d: Preparing
23135df75b44: Preparing
b43408d5f11b: Preparing
c761bb163c21: Waiting
d54e4a309a82: Waiting
fe6d8881187d: Waiting
23135df75b44: Waiting
711c3c5d494b: Waiting
b43408d5f11b: Waiting
ac4e7c2ae042: Waiting
44486128a8c5: Waiting
4f22e947a5a8: Waiting
5f70bf18a086: Waiting
e681618568c0: Waiting
175e0b2fdf31: Layer already exists
0dee4818e1b0: Layer already exists
99baa2214840: Layer already exists
ffa1fd9a60c2: Layer already exists
eaaf0853d3af: Layer already exists
4f22e947a5a8: Layer already exists
5f70bf18a086: Layer already ex


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


In [45]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)


NameError: name 'aiplatform' is not defined